In [46]:
!pip install -q pyyaml h5py

In [47]:
import os

In [48]:
import tensorflow as tf
from tensorflow import keras

In [49]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

In [50]:
train_labels = train_labels[:1000] # 트레인셋 처음부터 1000번째꺼만
test_labels = test_labels[:1000] # 테스트셋 처음부터 1000번째꺼만

In [51]:
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

# /255.0 : 0~1의 값으로 resizing

In [52]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# 일단 2개의 layer로 구성된 모델정의
# SpareseCategoricalCrossentropy : label이 정수일때 사용

In [53]:
model = create_model()

In [54]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_8 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


**체크포인트 저장**

체크포인트 콜백

In [55]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [56]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

# 가중치만 저장하는 콜백을 만듦

In [57]:
model.fit(train_images,
         train_labels,
         epochs=10,
         validation_data=(test_images,test_labels),
         callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - 1s 7ms/step - loss: 1.5856 - accuracy: 0.5027 - val_loss: 0.7136 - val_accuracy: 0.7880

Epoch 00001: saving model to training_1\cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.4581 - accuracy: 0.8753 - val_loss: 0.5218 - val_accuracy: 0.8410

Epoch 00002: saving model to training_1\cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 0.3117 - accuracy: 0.9249 - val_loss: 0.4744 - val_accuracy: 0.8490

Epoch 00003: saving model to training_1\cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2350 - accuracy: 0.9304 - val_loss: 0.4424 - val_accuracy: 0.8580

Epoch 00004: saving model to training_1\cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 0.1699 - accuracy: 0.9640 - val_loss: 0.4257 - val_accuracy: 0.8700

Epoch 00005: saving model to training_1\cp.ckpt
Epoch 6/10
32/32 [==============================] - 0s 

In [58]:
ls{checkpoint_dir}

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: B4F6-6A43

 C:\Users\kbs77\tensor\training_1 디렉터리

2021-03-13  오전 11:44    <DIR>          .
2021-03-13  오전 11:44    <DIR>          ..
2021-03-13  오전 11:44                71 checkpoint
2021-03-13  오전 11:44         4,886,697 cp.ckpt.data-00000-of-00001
2021-03-13  오전 11:44             1,222 cp.ckpt.index
               3개 파일           4,887,990 바이트
               2개 디렉터리  21,596,774,400 바이트 남음


In [59]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(acc) # accuracy 출력했더니 형편없음

32/32 - 0s - loss: 2.4557 - accuracy: 0.0710
0.07100000232458115


In [60]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print(acc) # 가중치 로드하니까 괜찮음

32/32 - 0s - loss: 0.4155 - accuracy: 0.8660
0.8659999966621399


콜백 매개변수

In [61]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 파일이름에 에포크 번호 매김

In [62]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=3) # 3번째 에폭마다 가중치 저장

In [63]:
model = create_model()

In [64]:
model.save_weights(checkpoint_path.format(epoch=0))

In [65]:
model.fit(train_images,
         train_labels,
         epochs=50,
         callbacks=[cp_callback],
         validation_data=(test_images, test_labels),
         verbose=0)

# 새로운 콜백으로 훈련


Epoch 00003: saving model to training_2\cp-0003.ckpt

Epoch 00006: saving model to training_2\cp-0006.ckpt

Epoch 00009: saving model to training_2\cp-0009.ckpt

Epoch 00012: saving model to training_2\cp-0012.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00018: saving model to training_2\cp-0018.ckpt

Epoch 00021: saving model to training_2\cp-0021.ckpt

Epoch 00024: saving model to training_2\cp-0024.ckpt

Epoch 00027: saving model to training_2\cp-0027.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00033: saving model to training_2\cp-0033.ckpt

Epoch 00036: saving model to training_2\cp-0036.ckpt

Epoch 00039: saving model to training_2\cp-0039.ckpt

Epoch 00042: saving model to training_2\cp-0042.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00048: saving model to training_2\cp-0048.ckpt


In [66]:
ls{checkpoint_dir}

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: B4F6-6A43

 C:\Users\kbs77\tensor\training_2 디렉터리

2021-03-13  오전 11:44    <DIR>          .
2021-03-13  오전 11:44    <DIR>          ..
2021-03-13  오전 11:44                81 checkpoint
2021-03-13  오전 11:44         1,628,730 cp-0000.ckpt.data-00000-of-00001
2021-03-13  오전 11:44               402 cp-0000.ckpt.index
2021-03-13  오전 11:44         4,886,697 cp-0003.ckpt.data-00000-of-00001
2021-03-13  오전 11:44             1,222 cp-0003.ckpt.index
2021-03-13  오전 11:44         4,886,697 cp-0006.ckpt.data-00000-of-00001
2021-03-13  오전 11:44             1,222 cp-0006.ckpt.index
2021-03-13  오전 11:44         4,886,697 cp-0009.ckpt.data-00000-of-00001
2021-03-13  오전 11:44             1,222 cp-0009.ckpt.index
2021-03-13  오전 11:44         4,886,697 cp-0012.ckpt.data-00000-of-00001
2021-03-13  오전 11:44             1,222 cp-0012.ckpt.index
2021-03-13  오전 11:44         4,886,697 cp-0015.ckpt.data-00000-of-00001
2021-03-13  오전 11:44             1,222 cp-0015.ckpt.index
2

In [67]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

# 마지막 체크포인트 선택 아래에서 쓸거

'training_2\\cp-0048.ckpt'

In [68]:
model = create_model()
model.load_weights(latest) # 마지막 체크포인트에서 불러옴
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print(acc)

32/32 - 0s - loss: 0.4892 - accuracy: 0.8730
0.8730000257492065


**수동으로 가중치 저장**

In [69]:
model.save_weights('./checkpoints/my_checkpoint')

In [70]:
model = create_model()

In [71]:
model.load_weights('./checkpoints/my_checkpoint')

In [72]:
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)

32/32 - 0s - loss: 0.4892 - accuracy: 0.8730


**전체 모델 저장**

In [73]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.6015 - accuracy: 0.5258
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4519 - accuracy: 0.8770
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2959 - accuracy: 0.9019
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2223 - accuracy: 0.9423
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1753 - accuracy: 0.9550


In [74]:
!mkdir -p saved_model
model.save('saved_model/my_model')

#전체 모델을 저장?

하위 디렉터리 또는 파일 -p이(가) 이미 있습니다.
다음 내용 진행 중 오류 발생: -p.
하위 디렉터리 또는 파일 saved_model이(가) 이미 있습니다.
다음 내용 진행 중 오류 발생: saved_model.


INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [75]:
ls saved_model
# 위에서 생성한 모델 3개 다 들어가 있음

SyntaxError: invalid syntax (<ipython-input-75-01f0a46f1ff7>, line 1)

In [76]:
ls saved_model/my_model

스위치가 틀립니다 - "my_model".


In [77]:
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

# 모델 가져옴

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [78]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4288 - accuracy: 0.8510
(1000, 10)


**HDF5 파일로 저장**

In [79]:
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('my_model.h5') # 전체 모델을 HDF5 파일로 저장

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.6150 - accuracy: 0.5040
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4521 - accuracy: 0.8593
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2825 - accuracy: 0.9346
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1923 - accuracy: 0.9670
Epoch 5/5
32/32 [==============================] - 0s 2ms/step - loss: 0.1601 - accuracy: 0.9632


In [80]:
new_model = tf.keras.models.load_model('my_model.h5')
new_model.summary() # 저장된 h5 파일 가져와서 보기

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_14 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [81]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)

32/32 - 0s - loss: 0.4382 - accuracy: 0.8560
